In [1]:
import numpy as np
from scipy import linalg
import os
import re
import linecache
import sys

import copy
import pp, time

from matplotlib import rc
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import AutoLocator
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
###################################################################
#
#  SIMPLE PROGRESS BAR
#
###################################################################
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))
    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

###################################################################
#
#  PARSING LATTICE
#
###################################################################
def parse_outcar_lattice(fname):
    step = 0
    tmp_ar = np.zeros((3, 3))
    angles = np.zeros((3, 1))
    vlen = np.zeros((3, 1))
    latt_dict = {}
    with open(fname) as f:
        for (i, line) in enumerate(f):
            if 'VOLUME and BASIS-vectors are now' in line:
                step += 1
                key = 'step' + str(step)
                latt_dict[key] = {}
                for (k, j) in enumerate(range(i + 6, i + 9)):
                    tmp_ar[k, :] = np.asarray(str(linecache.getline(fname, j)).replace(
                        '\n', '').split()[0:3], dtype=float)
                latt_dict[key]['lattice'] = np.copy(tmp_ar)
                for kk in range(0, 3):
                    vlen[kk] = np.sqrt(np.dot(tmp_ar[kk, :], tmp_ar.T[kk, :]))
                latt_dict[key]['lat_vetc_len'] = np.copy(vlen)
                latt_dict[key]['volume'] = float(
                    str(linecache.getline(fname, i + 4)).replace('\n', '').split()[4])
                angles[0] = vect_angle(
                    tmp_ar[2, :], tmp_ar[1, :])  # c*b, i.e. alpha
                angles[1] = vect_angle(
                    tmp_ar[2, :], tmp_ar[0, :])  # c*a, i.e. beta
                angles[2] = vect_angle(
                    tmp_ar[0, :], tmp_ar[1, :])  # a*b, i.e. gamma
                latt_dict[key]['angles'] = np.copy(angles)
    linecache.clearcache()
    return latt_dict


def mk_latt_3d_array(latt_dict):
    """
    Extraction of lattice parameters sotred into a dictionary
    into a 3D array with Z axis corresponding to to each step of MD
    """
    n_steps = len(latt_dict.keys())
    output = np.zeros((3, 3, n_steps))
    print('Exctrating lattice')
    for index in range(0, n_steps):
        key = 'step' + str(index + 1)
        output[:, :, index] = np.copy(latt_dict[key]['lattice'])
        progress(index+1, n_steps, '')
    print('\n')
    return output


def vect_angle(x, y):
    dot_prod = np.dot(x, y)
    norm_x = np.sqrt(np.dot(x, x.T))
    norm_y = np.sqrt(np.dot(y, y.T))
    angle = np.arccos(dot_prod / (norm_x * norm_y))
    return np.degrees(angle)

###################################################################
#
#  Text to array conversion
#
###################################################################
def extrat_num_from_list(lst):
    """
    Returns a NumPy 1D array from a list with mixed
    numerical and text elements
    """
    ff = []
    for element in lst:
        try:
            ff.append(float(element))
        except:
            pass
    return np.array(ff)


def list2array(out_lst):
    """
    Converts a list, which has elements with mixed extries of numbers and digits
    into a 2D NumPy array
    """
    tmp = extrat_num_from_list(out_lst[0].replace('\n','').replace(',','').replace('[','').replace(']','').split())
    out_array = np.zeros((len(out_lst),tmp.shape[0]))
    for row in range(out_array.shape[0]):
        out_array[row,:] = extrat_num_from_list(out_lst[row].replace('\n','').replace(',','').replace('[','').replace(']','').split())
    return out_array


def parse_stresses(path, fname):
    fopen = path + '/' + fname
    out_lst = []
    with open (fopen, 'r') as f:
        for line in f:
            if 'Total+' in line:
                out_lst.append(line)
    return list2array(out_lst)
###################################################################
#
#  COORDINATES MANIPULATIONS
#
###################################################################
def cart2frac_npt(pos_pbc, lattice, sc_atoms, ind_atom=None, partial_set=False):
    """
    Conversion of cartesian coordinates as read from OUTCAR to fractional
    performed with concern of variable lattice.
    """
    if not partial_set:
        n_atoms = sc_atoms.sum()
    else:
        n_atoms = sc_atoms[ind_atom]
    output = np.zeros(pos_pbc.shape)
    sep1 = 0
    sep2 = 0
    n_steps = pos_pbc.shape[0] / n_atoms
    for index in range(0, n_steps):
        sep1 = int(index * n_atoms)
        sep2 = int(sep1 + n_atoms)
        output[sep1:sep2, :] = np.copy(
            cart2frac(pos_pbc[sep1:sep2, :], lattice[:, :, index]))
        progress(index+1, n_steps, '')
    print('\n')
    return output


def frac2cart_npt(pos_pbc, lattice, sc_atoms, ind_atom=None, partial_set=False):
    """
    Conversion of fractional coordinates to cartesian ones
    performed with concern of variable lattice.
    """
    if not partial_set:
        n_atoms = sc_atoms.sum()
    else:
        n_atoms = sc_atoms[ind_atom]
    n_steps = pos_pbc.shape[0] / n_atoms
    output = np.zeros(pos_pbc.shape)
    sep1 = 0
    sep2 = 0
    for index in range(0, n_steps):
        sep1 = int(index * n_atoms)
        sep2 = int(sep1 + n_atoms)
        output[sep1:sep2, :] = np.copy(
            frac2cart(pos_pbc[sep1:sep2, :], lattice[:, :, index]))
        progress(index+1, n_steps, '')
    print('\n')
    return output


def cart2frac(dataset, a_lat):
    """
    This function covrets the cartesian atomic coordinates into fractional ones.
    """
    a_inv = linalg.inv(a_lat)
    return np.dot(dataset,a_inv)


def frac2cart(dataset, a_lat):
    """
    This function covrets the fractions atomic coordinates into cartesian ones.
    Returns:
            a 2d NumPy array of Cartesian coordites
    """
    return np.dot(dataset,a_lat)

def unwrap_PBC(coords,num_atoms):
    N_atoms = num_atoms.sum()
    dim = coords.shape
    n_steps = dim[0] / N_atoms
    for step in range(0, n_steps - 1):
        progress(step+1, dim[0] / N_atoms)
        sep1_this = step * N_atoms
        sep2_this = sep1_this + N_atoms
        this_step = coords[sep1_this:sep2_this, :]
        sep1_next = sep2_this
        sep2_next = sep1_next + N_atoms
        next_step = coords[sep1_next:sep2_next, :]
        check_cond = next_step - this_step
        for atom in range(0, N_atoms):
            for k in range(0, 3):
                if check_cond[atom, k] > 0.5:
                    indexes = range((step + 1) * N_atoms +
                                    atom, dim[0], N_atoms)
                    coords[indexes, k] -= 1
                elif check_cond[atom, k] < -0.5:
                    indexes = range((step + 1) * N_atoms +
                                    atom, dim[0], N_atoms)
                    coords[indexes, k] += 1
    return coords

def extract_atom_coordinates_2d(pos, sc_at, atoms_index):
    steps = pos.shape[0] / sc_at.sum()
    n_atoms_of_interest = pos.shape[0] / sc_at.sum() * sc_at[atoms_index]
    r = np.zeros((n_atoms_of_interest, 3))
    N_atoms = sc_at.sum()
    N_k_1 = 0
    if atoms_index > 0:
        for i in range(0, atoms_index):
            N_k_1 = N_k_1 + sc_at[i]
    else:
        N_k_1 = 0
    for i in range(1, steps + 1):
        index1 = (i - 1) * sc_at[atoms_index]
        index2 = i * sc_at[atoms_index]
        index1_pos = (i - 1) * sc_at.sum() + N_k_1
        index2_pos = (i - 1) * sc_at.sum() + N_k_1 + sc_at[atoms_index]
        r[int(index1):int(index2), 0:3] = pos[int(
            index1_pos):int(index2_pos), 0:3]
    return r
    
def extract_atom_coordinates_3d(pos,sc_at,ind_atom):
    dim = pos.shape
    n_steps = dim[0] / sc_at.sum()
    Ret = np.zeros((n_steps, 3, int(sc_at[ind_atom])))
    r1 = extract_atom_coordinates_2d(pos,sc_at,ind_atom)       
    print('\n')
    for i in range(0, n_steps):
        progress(i, n_steps)
        for j in range(0, Ret.shape[2]):
            Ret[i, :, j] = r1[i * sc_at[ind_atom] + j]
    print('\n')
    return Ret


###################################################################
#
#  WRITING DOWN POSCAR
#
###################################################################
def mk_text_for_POSCAR(sys_name, at_lst_names, sc_factor, SC_atoms):
    text_to_POSCAR = {}
    text_to_POSCAR['system_name'] = sys_name
    text_to_POSCAR['atoms_names_list'] = at_lst_names
    text_to_POSCAR['scaling_factor'] = str(sc_factor)
    text_to_POSCAR['number_atoms'] = SC_atoms
    return text_to_POSCAR

def write_POSCAR(fname, pos_step, lat_data, text_to_POSCAR,flag_direct = True):
    with open(fname, 'w') as f:
        f.write(text_to_POSCAR['system_name'] + '\n')
        f.write(str(text_to_POSCAR['scaling_factor']) + '\n')
        # append lattice
        np.savetxt(f, lat_data, fmt='%22.18f')
        # append atoms names and their numbers
        tmp = str(text_to_POSCAR['atoms_names_list'])
        symbols_to_replace = ['[', ']', ',', '\'']
        for i in range(0, len(symbols_to_replace)):
            tmp = tmp.replace(symbols_to_replace[i], '')
        f.write(tmp + '\n')
        symbols_to_replace = ['[', ']']
        tmp = str(text_to_POSCAR['number_atoms'])
        for i in range(0, len(symbols_to_replace)):
            tmp = tmp.replace(symbols_to_replace[i], '')
        f.write(tmp + '\n')
        # append positions
        if flag_direct:
            f.write('Direct' + '\n')
        else:
            f.write('Cartesian' + '\n')
        np.savetxt(f, pos_step, fmt='%22.18f')
    return

##############################################################################
#
#     Finds all atomsB which are neighbors to atomsA
#
##############################################################################
def mk_translation_vector():
    sc_dim = np.array([2, 2, 2])
    output = np.zeros((27, 3))
    counter = 0
    for i in np.arange(-1, 2):
        for j in np.arange(-1, 2):
            for k in np.arange(-1, 2):
                output[counter, :] = np.array([i, j, k])
                counter += 1
    return output
#
def extend_cell_by_tr_vect_dev(pos_step, tr_vect,latt):
    output = np.zeros((pos_step.shape[0], pos_step.shape[1], tr_vect.shape[0]))
    for (i, vect) in enumerate(tr_vect):
        output[:, :, i] = np.dot((pos_step + vect),latt)
    return output

def find_all_neighbours(posA, posB,lattice,min_bond, max_bond):
    """
    A general algorithm to search bonds between atoms A and B (in this order!)
    posA(B) -- fractional (direct) coordinates of the atoms A(B)
    lattice -- NumPy array of the lattice vectors
    min(max)_bond -- minimal(maximal) distance between the atoms
    RETURNS:
    a dictionary bonds_dict with the strucutre:
    |--'indeces': all possible inxeing is here
    |   |--'absolut' -- indexes in the concatenated array of posA, posB (in this sequence)
    |   |--'only_atoms_A(B)' -- indeces of the atoms A only in the array posA(B)
    |--'distance'
        |--'only_atoms_A(B)' --- distances between atoms A(B)
    """
    coords = np.append(posA, posB,axis=0)
    translation_vect = mk_translation_vector()
    extended_cell = extend_cell_by_tr_vect_dev(coords,translation_vect,lattice)
    # indexes of neighbours of given atom
    bonds_dict = {}
    for AtomI in range(0,posA.shape[0]):
        dr_extended_cell = extend_cell_by_tr_vect_dev(coords-coords[AtomI,:],translation_vect,lattice)
        dr_norm = np.linalg.norm(dr_extended_cell, axis=1,keepdims=True)
        nn_arr = np.argwhere(np.logical_and(dr_norm>= min_bond, dr_norm <= max_bond))
        bonds_dict[AtomI] = {}
        bonds_dict[AtomI]['indeces'] = {}
        bonds_dict[AtomI]['indeces']['absolut'] = nn_arr[:,0]
        bonds_dict[AtomI]['indeces']['only_atoms_A'] = nn_arr[:,0][nn_arr[:,0]<=posA.shape[0]-1]
        bonds_dict[AtomI]['indeces']['only_atoms_B'] = nn_arr[:,0][nn_arr[:,0]>=posA.shape[0]-1]-posA.shape[0]
        bonds_dict[AtomI]['distances'] = {}
        flag_AA = nn_arr[:,0]<=posA.shape[0]-1
        flag_BB = nn_arr[:,0]>=posA.shape[0]-1
        bonds_dict[AtomI]['distances']['only_atoms_A'] = dr_norm[nn_arr[:,0][flag_AA],:,nn_arr[:,2][flag_AA]]
        bonds_dict[AtomI]['distances']['only_atoms_B'] = dr_norm[nn_arr[:,0][flag_BB],:,nn_arr[:,2][flag_BB]]
        bonds_dict[AtomI]['dr'] = np.copy(dr_extended_cell[nn_arr[:,0],:,nn_arr[:,2]])
    return bonds_dict


def complexes_extract(pos_A,pos_B,lattice,ligands_per_center,min_AB,max_AB):
    """
    Extracts dr=ligands_for_pos_A(AtomNum)-pos_A(AtomNum) for each MD time step.
    The function assumes that for each time step number of ligands for each central atom A
    is constant and all A atoms have the same number of ligands
    """
    num_steps = pos_A.shape[0]
    NH_complexes = np.zeros((num_steps, ligands_per_center, 3))
    AB_dr_dict = {}
    for i in range(pos_A.shape[2]):
        AB_dr_dict[i] = np.copy(NH_complexes)
    for step in range(num_steps):
        pos_A_snapshot = pos_A[step,:,:].T
        pos_B_snapshot = pos_B[step,:,:].T
        bonds_dict_AB = find_all_neighbours(pos_A_snapshot,pos_B_snapshot,lattice[:,:,step],min_AB,max_AB)
        for AtomNum in range(len(bonds_dict_AB.keys())):
            AB_dr_dict[AtomNum][step,:,:] = bonds_dict_AB[AtomNum]['dr']
        progress(step+1,num_steps)
    return AB_dr_dict


def frac2cart_var_latt(pos, lattice):
    pos_cart = np.zeros(pos.shape)
    for i in range(pos.shape[0]):
        pos_cart[i,:,:] = frac2cart(pos[i,:,:].T,lattice[:,:,i]).T
    return pos_cart

def connected_components(graph):
    """
    This finds connected components in undirected graph, which must be provided in symmetric form 
    (both edge x to y and y to x always given)
    The graph is specified as dict:
    graph[vertex] is set(all edges of vertex)
    
    """
    neighbors = graph
    seen = set()
    def component(node):
        queue = set([node])
        result = list()
        while queue:
            node = queue.pop()
            seen.add(node)
            #for x in neighbors[node]:
            #    if x not in seen:
            #        queue.append(x)
            queue |= set(neighbors[node]) - seen
            result.append(node)
        return result
    all_result = list()
    for node in neighbors:
        if node not in seen:
            all_result.append(component(node))
    return all_result

def implement_PBC (data__):
    data = np.copy(data__)
    for i in range(data.shape[0]):
        for j in range(3):
            if data[i,j]>=1:
                data[i,j]-=1
            if data[i,j]<0:
                data[i,j]+=1
    return data


def segregate_by_planes(positions, plane_idx,tolerance=1e-02):
    """
    Returns a list with indeces of atoms
    lying in the same planes (XY, ZY,XZ) 
    defined by plane_idx:
    -- XY : 0
    -- YZ : 1
    -- XZ : 2
    The input data is 2D!
    """
    if np.ndim(positions) > 2:
        raise ValueError('The input coordinates must be 2D array!')
    else:
        idx_dict = {}
        for i in range(positions.shape[0]):
            condition = np.isclose(positions[:,plane_idx] -positions[i,plane_idx],0,atol=tolerance)
            idx_dict[i] = set(np.argwhere(condition==True).flatten()) # build a connected graph
        return connected_components(idx_dict)
    
    
def round_positions(_pos_,tolerance=1e-2):
    output = np.zeros(_pos_.shape, dtype=float)
    for plane_idx in range(3):
        idx_list = segregate_by_planes(_pos_,plane_idx, tolerance=tolerance)
        for j in range(len(idx_list)):
            idx_ = idx_list[j]
            plane_idx_coords = _pos_[idx_][:,plane_idx]
            common_coord = np.round(min(plane_idx_coords*(1/tolerance*10)))/(1/tolerance*10)
            for i in range(len(idx_)):
                output[idx_[i],plane_idx] = common_coord
    return output


def exctrac_UC_coordinates_from_SC(array__, Nxyz):
    cnt = 0
    uc_from_sc_dict = {}
    array = np.copy(array__)
    condition_less = np.array([0.5,1.0])
    condition_more = np.array([-0.5,0.5])
    for i in range(Nxyz[0]):
        for j in range(Nxyz[1]):
            for k in range(Nxyz[2]):
                idx_less_x = np.argwhere(array[:,0] < condition_less[i])
                idx_more_x = np.argwhere(array[:,0] >= condition_more[i])
                idx_less_y = np.argwhere(array[:,1] < condition_less[j])
                idx_more_y = np.argwhere(array[:,1] >= condition_more[j])            
                idx_less_z = np.argwhere(array[:,2] < condition_less[k])
                idx_more_z = np.argwhere(array[:,2] >= condition_more[k])
                idx_x = np.intersect1d(idx_less_x,idx_more_x)
                idx_y = np.intersect1d(idx_less_y,idx_more_y)
                idx_z = np.intersect1d(idx_less_z,idx_more_z)
                idx_xy = np.intersect1d(idx_x,idx_y)
                idx_xyz = np.intersect1d(idx_xy,idx_z)
                uc_from_sc_dict[cnt] = np.copy(idx_xyz)
                cnt +=1
    return uc_from_sc_dict

def extract_H_at_centers(dict_,arr_idxs,arr_centers):
    for i in range(arr_idxs.shape[0]):
        idx = arr_idxs[i]
        dH_coords = dict_[idx]['dr']
        if i == 0 :
            H_N_cart = arr_centers[i,:] + dH_coords
        else:
            H_N_cart = np.append(H_N_cart,arr_centers[i,:] + dH_coords, axis=0)
    return H_N_cart

In [4]:
import ase.io

In [ ]:
contcar_name = 'CONTCAR_skip_85_0GPa'
poscar_name = 'POSCAR_skip_85'
NumAtoms = np.array([32,32,256])
name_atoms = ['N','B','H']
contcar = ase.io.read(contcar_name)
poscar = ase.io.read(poscar_name)
#
init_pos = poscar.get_scaled_positions()
contcar_pos = contcar.get_scaled_positions()
new_lattice = contcar.get_cell()
init_lattice = poscar.get_cell()

In [ ]:
# unwrap PBC for the relaxed structure
unwrapped = unwrap_PBC(np.append(init_pos,contcar_pos, axis=0),NumAtoms)
unwrapped_new = unwrapped[:NumAtoms.sum(),:]

In [ ]:
# extract coordinates from POSCAR
new_N_direct = unwrapped_new[0:NumAtoms[0],:]
new_B_direct = unwrapped_new[NumAtoms[0]:NumAtoms[0:2].sum(),:]
new_H_direct = unwrapped_new[NumAtoms[0:2].sum():,:]

In [ ]:
# indetify the complexes in the new structure
min_NH = 0.1
max_NH = 1.7
ligand_num = 4
new_NH = find_all_neighbours(new_N_direct,new_H_direct,new_lattice,min_NH,max_NH)
new_BH = find_all_neighbours(new_B_direct,new_H_direct,new_lattice,min_NH,max_NH)

In [ ]:
Nxyz = np.array([2,2,2])

In [ ]:
# extract indexes of atoms in the SC
N_UC_dict = exctrac_UC_coordinates_from_SC(new_N_direct,Nxyz)
B_UC_dict = exctrac_UC_coordinates_from_SC(new_B_direct,Nxyz)

In [ ]:
B_UC_dict

In [ ]:
N_UC_dict

In [ ]:
# save as POSCARs all the unit cells of the SC
new_lattice_UC = np.zeros(new_lattice.shape)
# scale the lattice to the UC
for i in range(3):
    new_lattice_UC[:,i] = new_lattice[:,i] / Nxyz[i]
for SC_key in N_UC_dict.keys():
    # indeces:
    N_UC_idx = N_UC_dict[SC_key]
    B_UC_idx = B_UC_dict[SC_key]
    # conver to the Cartesian coordinates
    N_UC_cart = frac2cart(new_N_direct[N_UC_idx,:],new_lattice)
    B_UC_cart = frac2cart(new_B_direct[B_UC_idx,:],new_lattice)
    # find corrdinates of the H atoms
    H_N_cart = extract_H_at_centers(new_NH,N_UC_idx,N_UC_cart)
    H_B_cart = extract_H_at_centers(new_BH,B_UC_idx,B_UC_cart)
    # append everything
    H_cart = np.append(H_N_cart,H_B_cart, axis=0)
    NB_cart = np.append(N_UC_cart,B_UC_cart, axis=0)
    UC_all_atoms_cart = np.append(NB_cart,H_cart,axis=0)


    # convert into the direct coordinates of the scaled lattice
    UC_all_atoms_direct = cart2frac(UC_all_atoms_cart, new_lattice_UC)
    UC_all_atoms_direct = implement_PBC(UC_all_atoms_direct)
    system_name='4*NH4_4BH4_' + str(SC_key) + '_from_SC'
    fname = 'POSCAR_' + str(SC_key)
    num_at = np.array([N_UC_cart.shape[0],B_UC_cart.shape[0],H_cart.shape[0]])
    txt2POSCAR = mk_text_for_POSCAR(system_name,name_atoms,1.0,num_at)
    write_POSCAR(fname ,UC_all_atoms_direct,new_lattice_UC,txt2POSCAR,True)
